In [56]:
"""
Simple example using LSTM recurrent neural network to classify IMDB
sentiment dataset.
References:
    - Long Short Term Memory, Sepp Hochreiter & Jurgen Schmidhuber, Neural
    Computation 9(8): 1735-1780, 1997.
    - Andrew L. Maas, Raymond E. Daly, Peter T. Pham, Dan Huang, Andrew Y. Ng,
    and Christopher Potts. (2011). Learning Word Vectors for Sentiment
    Analysis. The 49th Annual Meeting of the Association for Computational
    Linguistics (ACL 2011).
Links:
    - http://deeplearning.cs.cmu.edu/pdfs/Hochreiter97_lstm.pdf
    - http://ai.stanford.edu/~amaas/data/sentiment/
"""
from __future__ import division, print_function, absolute_import
import os
import numpy as np
import sklearn.preprocessing as prep
import librosa
from keras.layers import Dense, Dropout, LSTM, Embedding
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential

# IMDB Dataset loading
#train, test, _ = imdb.load_data(path='imdb.pkl', n_words=10000,
#                                valid_portion=0.1)

print("Imports doned")
def wav2mfcc(file_path, max_pad_len=10):
    wave, sr = librosa.load(file_path, mono=True, sr=None)
    print("file read=",file_path)
    print(wave.shape,sr)
    wave=prep.maxabs_scale(wave) #Normalize amplitude to +-1
    mfcc = librosa.feature.mfcc(wave, sr=sr).transpose()
    #print(mfcc.shape)
    #print(mfcc[1,:5])
    dif=max_pad_len-mfcc.shape[0]
    #padding sample
    if dif>0:
        #fill holes
        #print("filling holes")
        for i in range(dif):
            mfcc=np.vstack([mfcc,np.zeros(mfcc.shape[1])])
    else:
        #take away excess
        #print("excess")
        mfcc=mfcc[:max_pad_len]
    #print(mfcc.shape)
    #print(mfcc[199,:5])
    return mfcc

class EasyASR(object):
    '''
    classdocs
    '''
    def __init__(self, filePath="/home/francisco/voz"):
        '''
        Constructor
        '''
        basepath=filePath
        self.mfccs=[]
        self.speakers=[]
        self.words=[]
        fNames=os.listdir(basepath)
        print("fNames=",len(fNames))
        for i,fn in enumerate(fNames):
            #file name format speaker_word_number.wav
            n,_=fn.split(".")#take away extension .wav
            mfccs=wav2mfcc(basepath+'/{}.wav'.format(n))
            self.mfccs.append(mfccs)
            #decode file name format speaker_word_number
            speaker,word,number=n.split("_")
            self.speakers.append(speaker)
            self.words.append(word)
        leSpeakers=prep.LabelEncoder()
        leSpeakers.fit(self.speakers)
        leSpeakers.transform(self.speakers)
        self.leWords=prep.LabelEncoder()
        self.leWords.fit(self.words)
        print(len(self.words))
        print(self.words)
        self.iLabels=self.leWords.transform(self.words)

ea=EasyASR()
print(ea.leWords)

Imports doned
fNames= 108
file read= /home/francisco/voz/paco_no_006.wav
(3871,) 16000
file read= /home/francisco/voz/paco_si_008.wav
(4963,) 16000
file read= /home/francisco/voz/paco_dos_002.wav
(9285,) 16000
file read= /home/francisco/voz/paco_uno_004.wav
(4434,) 16000
file read= /home/francisco/voz/paco_no_007.wav
(2428,) 16000
file read= /home/francisco/voz/paco_para_003.wav
(4882,) 16000
file read= /home/francisco/voz/paco_cuatro_003.wav
(6972,) 16000
file read= /home/francisco/voz/paco_nueve_001.wav
(6524,) 16000
file read= /home/francisco/voz/carlos_si_002.wav
(4630,) 16000
file read= /home/francisco/voz/paco_buenos-dias_001.wav
(14450,) 16000
file read= /home/francisco/voz/paco_buenos-dias_003.wav
(14816,) 16000
file read= /home/francisco/voz/irene_hola_003.wav
(6847,) 16000
file read= /home/francisco/voz/paco_seis_001.wav
(9176,) 16000
file read= /home/francisco/voz/paco_buenos-dias_007.wav
(8618,) 16000
file read= /home/francisco/voz/paco_ocho_001.wav
(6462,) 16000
file read=

In [49]:
trainX, trainY = np.array(ea.mfccs),np.array(ea.iLabels)
#testX, testY = test

maxlen=trainX.shape[1]
lstm_hidden_units=128
input_dim=trainX.shape[2]
outputs=max(trainY)+1

print(outputs)

# Converting labels to binary vectors
trainY = to_categorical(trainY,outputs)
#testY  = to_categorical(testY, outputs)
print (trainX.shape,trainY.shape)
print(trainY)

# Network building
model = Sequential()
model.add(LSTM(128, input_dim=input_dim, input_length=maxlen))
model.add(Dropout(0.65))
model.add(Dense(outputs,activation="softmax"))
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])


17
(108, 10, 20) (108, 17)
[[ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  1.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  1. ...,  0.  0.  0.]]


In [50]:
# Training
history=model.fit(trainX, trainY, batch_size=2,nb_epoch=50)

Epoch 1/50
108/108 [==============================] - 1s - loss: 3.2700 - acc: 0.0278          
Epoch 2/50
108

In [60]:
print(trainX[5:7,:,:].shape)
y_pred=np.argmax(model.predict(trainX[5:6,:,:]))
number=ea.leWords.inverse_transform(y_pred)
print(number)
print(ea.leWords.classes_)

(2, 10, 20)
para
['avanza' 'buenos-dias' 'casa' 'cinco' 'cuatro' 'dos' 'hola' 'no' 'nueve'
 'ocho' 'para' 'retrocede' 'seis' 'si' 'siete' 'tres' 'uno']
